In [ ]:
###  SageMaker
###
###
###  Template creates SageMaker env:
#
#      1. Create AWS Code Repository from github
#
#      2. Create AWS Notebook Instance --- connect to github repository. Creates an Amazon SageMaker notebook instance. A notebook instance is a machine learning (ML) compute instance running on a Jupyter notebook.
#         In a CreateNotebookInstance request, specify the type of ML compute instance that you want to run. Amazon SageMaker launches the instance, installs common libraries that you can use to explore datasets for model training, and attaches an ML storage volume to the notebook instance.
#         Amazon SageMaker also provides a set of example notebooks. Each notebook demonstrates how to use Amazon SageMaker with a specific algorithm or with a machine learning framework.
#        
#         After receiving the request, Amazon SageMaker does the following:
#           a. Creates a network interface in the Amazon SageMaker VPC.
#           b. (Option) If you specified SubnetId , Amazon SageMaker creates a network interface in your own VPC, which is inferred from the subnet ID that you provide in the input. When creating this network interface, Amazon SageMaker attaches the security group that you specified in the request to the network interface that it creates in your VPC.
#           c. Launches an EC2 instance of the type specified in the request in the Amazon SageMaker VPC. If you specified SubnetId of your VPC, Amazon SageMaker specifies both network interfaces when launching this instance. This enables inbound traffic from your own VPC to the notebook instance, assuming that the security groups allow it.
#      
#         After creating the notebook instance, Amazon SageMaker returns its Amazon Resource Name (ARN). You can't change the name of a notebook instance after you create it.
#
#      3. Create S3 Training bucket and  Model artifacts bucket
#
#      4. Upload training file to bucket
#
#      5. Train - Python Built-in  or Container  -- Store Model artifact in S3 Model artifact bucket
#  
#      6. Model Deploy - Python
#
#      7. Create API Gateway point to Lambda Function
#
#      8. Create Lambda Function to execute SageMaker Endpoint
#  
#      9. Postman to call API Gateway and return results




In [2]:
import boto3

client = boto3.client('sagemaker')

In [1]:
###  #1 AWS Code Respository to link to data science code base
#
#        a.  Need to use AWS Secrets Manager to create Secret Name --- ARN is available by clicking on Secret Name
#
#        b.  Point to URL of github with code
#

response = client.create_code_repository(
    CodeRepositoryName='sm-git-dev-repository',
    GitConfig={
        'RepositoryUrl': 'https://github.com/toadies/7346_Cloud_Automation',
        'Branch': 'master',
        'SecretArn': 'git-secret'
    }
)
response

NameError: name 'client' is not defined

In [1]:
###  #1 AWS Code Respository to link to data science code base
#
#        a.  Need to use AWS Secrets Manager to create Secret Name --- ARN is available by clicking on Secret Name
#
#        b.  Point to URL of github with code
#

import boto3

client = boto3.client('sagemaker')

response = client.create_code_repository(
    CodeRepositoryName='sm-git-dev-repository',  ## arg
    GitConfig={
        'RepositoryUrl': 'https://github.com/toadies/7346_Cloud_Automation',  ##arg
        'Branch': 'master',
        'SecretArn': 'arn:aws:secretsmanager:us-east-1:697525789718:secret:git-secret-NnkzF8'  
    }
)
response


{'CodeRepositoryArn': 'arn:aws:sagemaker:us-east-1:697525789718:code-repository/sm-git-dev-repository',
 'ResponseMetadata': {'RequestId': 'df8e8b10-0cdb-42f8-9afb-68f1d69b5f5a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'df8e8b10-0cdb-42f8-9afb-68f1d69b5f5a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Sun, 03 Nov 2019 22:52:18 GMT'},
  'RetryAttempts': 0}}

In [2]:
## Create SageMaker Notebook Instance  --- training code and deploy will run from github repository

response = client.create_notebook_instance(
    NotebookInstanceName='sm-dev-iris-1',  ## arg 
    InstanceType='ml.t2.medium',  ## arg
    RoleArn='arn:aws:iam::697525789718:role/service-role/AmazonSageMaker-ExecutionRole-20191019T150576',
    DirectInternetAccess='Enabled',
    VolumeSizeInGB=5,
    AcceleratorTypes=[
        'ml.eia1.medium',
    ],
    DefaultCodeRepository='sm-git-dev-repository',  ## chg to field name from above respository code
    RootAccess='Enabled'
)

response

{'NotebookInstanceArn': 'arn:aws:sagemaker:us-east-1:697525789718:notebook-instance/sm-dev-iris-1',
 'ResponseMetadata': {'RequestId': '96883aec-ddac-4148-8fb4-191536efbe6a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '96883aec-ddac-4148-8fb4-191536efbe6a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Sun, 03 Nov 2019 22:53:32 GMT'},
  'RetryAttempts': 0}}

In [1]:
###   Use Chris' code to create S3 buckets  

import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

def create_bucket(bucket_prefix, s3_connection):
    session = boto3.session.Session()
    current_region = session.region_name
    # Create Configurations
    configs = {}
    if current_region != "us-east-1":  #arg
        config["LocationConstraint"] = current_region

    bucket_name = create_bucket_name(bucket_prefix)
    
    if current_region == "us-east-1":  #arg
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name)
    else:
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint":current_region})
        
    print(bucket_name, current_region)
    return bucket_name, bucket_response

In [2]:
###  Use Chris' code to create "Training" bucket

import boto3

s3_resource = boto3.resource('s3')
bucket_name, first_response = create_bucket(
    bucket_prefix='iris-train', 
    s3_connection=s3_resource.meta.client)

iris-trainb3119a0e-42d2-4ac4-a9e8-f3458356f096 us-east-1


In [3]:
###  Use Chris' code to create "Model Artifacts" bucket

import boto3

s3_resource = boto3.resource('s3')
bucket_name, first_response = create_bucket(
    bucket_prefix='iris-model-artifacts', 
    s3_connection=s3_resource.meta.client)

iris-model-artifactsd506bcfd-74e6-437c-aefb-5c74b6dacd24 us-east-1
